In [ ]:
!pip install tensorflow==2.4.0
!pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import tensorflow_model_optimization as tfmot

In [2]:
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [34]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data(label_mode='coarse')

train_images = ((train_images / 255.0) - 0.1307)/0.3081
test_images = ((test_images / 255.0) - 0.1307)/0.3081
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [48]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(92, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(92, (1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(92, (1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D((1, 1), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())


model.add(layers.Dense(20, activation='softmax'))


In [51]:
opt = tf.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
it_train = datagen.flow(train_images, train_labels, batch_size=64)
steps = int(train_images.shape[0] / 64)

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:0'):
	history = model.fit(it_train, steps_per_epoch=steps, epochs=50, validation_data=(test_images, test_labels))

In [ ]:
opt = tf.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
	history = model.fit(it_train, steps_per_epoch=steps, epochs=20, validation_data=(test_images, test_labels))

In [ ]:
model.save("network.h5")

In [ ]:
!wget https://github.com/darkyfoxy/AI_on_STM32/raw/main/network_CIFAR100/network.h5

In [49]:
model.load_weights("/content/network.h5")

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(model, **clustering_params)


opt = tf.optimizers.SGD(learning_rate=0.001, momentum=0.9)
clustered_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

clustered_model.fit(train_images, train_labels, batch_size=500, epochs=3, validation_data=(test_images, test_labels))

In [ ]:
clustered_model.evaluate(test_images, test_labels)

In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
model_no_quant_tflite = converter.convert()

open("clustered_network_without_optim.tflite", "wb").write(model_no_quant_tflite)

In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
def representative_dataset():
  for i in range(500):
    yield([test_images[i].reshape(1, 32, 32, 3).astype(np.float32)])

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
model_tflite = converter.convert()

open("clustered_network_FIQ_int_only_IIOT.tflite", "wb").write(model_tflite)